In [1]:
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
from pathlib import Path
import cv2
import numpy as np
import pandas as pd

from pathlib import Path
from Utils.utils import generate_and_save_denoised_images

In [ ]:
def main(clean_images_dir, noisy_images_dir, output_dir):

    clean_images_path = Path(clean_images_dir)
    noisy_images_path = Path(noisy_images_dir)
    output_path = Path(output_dir)

    output_path.mkdir(parents=True, exist_ok=True)

    rows = []
    for clean_image_path in clean_images_path.glob('*.png'):
        print(f"Processing {clean_image_path}...")
        noisy_image_path = list(noisy_images_path.glob(f"{clean_image_path.stem}*"))[0]
        if not noisy_image_path.exists():
            print(f"Noisy image for {clean_image_path} not found, skipping.")
            continue
        denoised_image_paths = generate_and_save_denoised_images(str(noisy_image_path), str(output_path))
        
        clean_image = cv2.imread(str(clean_image_path), cv2.IMREAD_COLOR)
        clean_image = cv2.cvtColor(clean_image, cv2.COLOR_BGR2RGB)
        
        row = {}
        for name, path in (denoised_image_paths | {"noisy": noisy_image_path}).items():
            image = cv2.imread(str(path), cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            l1 = np.mean(np.abs(     - image))
            l2 = np.mean((clean_image - image) * (clean_image - image))
            row[f"{name}_l1"] = l1
            row[f"{name}_l2"] = l2
        rows.append(row)
        
    df = pd.DataFrame(rows)
    df.to_csv(output_path / 'denoising_results.csv', index=False)

In [ ]:
clean_images_dir = Path("../Data/Raw/Just_one")
noisy_images_dir = Path("../Data/Transformed/Just_one")
    
main(clean_images_dir, noisy_images_dir, Path("outptut"))

Processing ..\Data\Raw\Just_one\circle_1.png...
✓ Completed: mmse_quad
✓ Completed: mmse_potts
✓ Completed: mape_sa_quad
✓ Completed: mape_sa_potts
Processing ..\Data\Raw\Just_one\circle_10.png...
✓ Completed: mape_sa_quad
✓ Completed: mmse_quad
✓ Completed: mape_sa_potts
✓ Completed: mmse_potts
Processing ..\Data\Raw\Just_one\circle_2.png...
